<hr>
<div style="background-color: lightgray; padding: 20px; color: black;">
<div>
<img src="https://th.bing.com/th/id/R.3cd1c8dc996c5616cf6e65e20b6bf586?rik=09aaLyk4hfbBiQ&riu=http%3a%2f%2fcidics.uanl.mx%2fwp-content%2fuploads%2f2016%2f09%2fcimat.png&ehk=%2b0brgMUkA2BND22ixwLZheQrrOoYLO3o5cMRqsBOrlY%3d&risl=&pid=ImgRaw&r=0" style="float: right; margin-right: 30px;" width="200"/> 
<font size="6.9" color="8C3061"><b>Curso de Optimización</b></font> <br>
<font size="4.5" color="8C3061"><b>Tarea 6 - Gradiente Conjugado</b></font> 
</div>
<div style="text-align: left">  <br>
Edison David Serrano Cárdenas. <br>
MSc en Matemáticas Aplicadas <br>
CIMAT - Sede Guanajuato <br>
</div>

</div>
<hr>


# <font color="8C3061" >**Cargar Librerías**</font> 

In [1]:
# load libraries
from typing import Tuple, Callable
import numpy as np
import scipy.optimize as opt

# load module Opti_functions
from opti_functions import Opti_functions as opti

# load visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

---

# <font color="8C3061" >**Ejercicio 1 (3.0 puntos)**</font>

1. Programe el método de gradiente conjugado lineal, Algoritmo 1 de la Clase 18,
   para resolver el sistema de ecuaciones $\mathbf{A}\mathbf{x}=\mathbf{b}$, donde
   $\mathbf{A}$ es una matriz simétrica y definida positiva.
   
   Haga que la función devuelva el último punto  $\mathbf{x}_k$, 
   el último residual $\mathbf{r}_k$, el número de iteraciones $k$ 
   y una variable binaria $bres$ que indique si se cumplió el criterio
   de paro ($bres=True$) o si el algoritmo terminó por
   iteraciones ($bres=False$).

2. Pruebe el algoritmo para resolver el sistema de ecuaciones 

$$ \mathbf{A}_1\mathbf{x}=\mathbf{b}_1$$

  donde

$$ \mathbf{A}_1 = n\mathbf{I} + \mathbf{1} = 
\left[\begin{array}{llll} n      & 0      & \cdots & 0 \\
                       0      & n      & \cdots & 0 \\ 
                       \vdots & \vdots & \ddots & \vdots \\
                       0      & 0      & \cdots & n \end{array}\right]
+ \left[\begin{array}{llll} 1    & 1      & \cdots & 1 \\
                       1      & 1      & \cdots & 1 \\ 
                       \vdots & \vdots & \ddots & \vdots \\
                       1      & 1      & \cdots & 1 \end{array}\right],  \qquad
\mathbf{b}_1 = \left[\begin{array}{l} 1 \\ 1 \\ \vdots \\ 1 \end{array}\right], $$

$n$ es la dimensión de la variable independiente
$\mathbf{x}=(x_1, x_2, ..., x_n)$, 
$\mathbf{I}$ es la matriz identidad y $\mathbf{1}$ es la matriz llena de 1's,
ambas de tamaño $n$.

También aplique el algoritmo para resolver el sistema 

$$ \mathbf{A}_2\mathbf{x}=\mathbf{b}_2$$

donde  $\mathbf{A}_2 = [a_{ij}]$ con

$$ a_{ij} = exp\left(-0.25(i-j)^2 \right),  \qquad
\mathbf{b}_2 = \left[\begin{array}{l} 1 \\ 1 \\ \vdots \\ 1 \end{array}\right] $$


- Use $\mathbf{x}_0$ como el vector cero, el máximo número de iteraciones 
  $N=n$ y una toleracia $\tau=\sqrt{n} \epsilon_m^{1/3}$,
  donde $\epsilon_m$ es el épsilon máquina.
- Pruebe el algoritmo resolviendo los dos sistemas de ecuaciones con $n=10, 100, 1000$ y 
  en cada caso imprima la siguiente información

- la dimensión $n$,
- el  número $k$ de iteraciones realizadas,
- las primeras y últimas 4 entradas del punto $\mathbf{x}_k$ que devuelve el algoritmo,
- la norma del residual $\mathbf{r}_k$, 
- la variable $bres$ para saber si el algoritmo puedo converger.

**Solución:**

1. Implementación del método de gradiente conjugado lineal:

In [2]:
def conjugate_gradient(x0:np.ndarray, 
                       A:np.ndarray,
                       b:np.ndarray, 
                       max_iter:int, 
                       tol:float) -> Tuple[np.ndarray, np.ndarray, int, bool]:
    """
    Conjugate gradient method to solve the linear system Ax = b
    
    :param x0: initial guess (np.ndarray)
    :param A: matrix A (np.ndarray)
    :param b: vector b (np.ndarray)
    :param max_iter: maximum number of iterations (int)
    :param tol: tolerance (float)
    
    :return: Tuple with the solution of the linear system, the residual, the number of iterations and a boolean indicating if the method converges
    """
    
    r0 = A @ x0 - b
    p0 = -r0
    
    for i in range(max_iter):
        if np.linalg.norm(r0) < tol:
            return x0, r0, i, True
        alpha = r0.T@r0 / (p0.T @ A @ p0)
        xk = x0 + alpha * p0
        rk = r0 + alpha * A @ p0
        beta = (rk.T @ rk) / (r0.T @ r0)
        pk = -rk + beta * p0
        r0 = rk
        p0 = pk
        x0 = xk
    return x0, r0, i, False  

2. Evaluación del algoritmo:

In [3]:
def matrix1(n:int) -> np.ndarray:
    return n*np.eye(n) + np.ones((n,n))

def matrix2(n:int) -> np.ndarray:
    return np.fromfunction(lambda i, j: np.exp(-0.25*(i - j)**2), (n, n))

Testear el algoritmo:

In [4]:
def test_conjugate_gradient(n:int,matrix:callable) -> None:
    """
    Test the conjugate gradient method
    
    :param n: size of the matrix
    """
    x0 = np.zeros(n)
    tol  = np.sqrt(n)*(np.finfo(float).eps)**(1/3)
    xk, rk, n_iter, success = conjugate_gradient(x0, matrix(n), np.ones(n), n, tol)

    print(f"Dimensión:\t\t{n}")
    print(f"Número de iteraciones:\t{n_iter}")
    print("Solución:\t\t",','.join(map(str, xk[:4])),",...,",','.join(map(str, xk[-4:])))
    print("Convergencia:\t\t", success,"\n") 

**Matriz 1:** Los resultados siguientes son esperados debido a que el vector $\mathbf{b}_1$ es un vector propio de $\mathbf{A}_1$, luego, el algoritmo converge en una iteración.

In [5]:
test_conjugate_gradient(10,matrix=matrix1)
test_conjugate_gradient(100,matrix=matrix1)
test_conjugate_gradient(1000,matrix=matrix1)

Dimensión:		10
Número de iteraciones:	1
Solución:		 0.05,0.05,0.05,0.05 ,..., 0.05,0.05,0.05,0.05
Convergencia:		 True 

Dimensión:		100
Número de iteraciones:	1
Solución:		 0.005,0.005,0.005,0.005 ,..., 0.005,0.005,0.005,0.005
Convergencia:		 True 

Dimensión:		1000
Número de iteraciones:	1
Solución:		 0.0005,0.0005,0.0005,0.0005 ,..., 0.0005,0.0005,0.0005,0.0005
Convergencia:		 True 



**Matriz 2:**

In [6]:
test_conjugate_gradient(10,matrix=matrix2)
test_conjugate_gradient(100,matrix=matrix2)
test_conjugate_gradient(1000,matrix=matrix2)

Dimensión:		10
Número de iteraciones:	5
Solución:		 1.3690991585471464,-1.166376817178349,1.6090828050291892,-0.6133905258859524 ,..., -0.6133905258860959,1.6090828050289954,-1.1663768171785018,1.3690991585470598
Convergencia:		 True 

Dimensión:		100
Número de iteraciones:	99
Solución:		 1.446307563888463,-1.4163725105090534,2.1104266146806023,-1.4249577511636988 ,..., -1.4249578827801062,2.110458023278651,-1.4163439459655913,1.446290949265481
Convergencia:		 False 

Dimensión:		1000
Número de iteraciones:	262
Solución:		 1.446288242264973,-1.4163595431324436,2.11051809682915,-1.4250723051291694 ,..., -1.4250723051291925,2.110518096829142,-1.4163595431324532,1.4462882422649672
Convergencia:		 True 



---

# <font color="8C3061" >**Ejercicio 2 (3.5 puntos)**</font>

Programar el método de gradiente conjugado no lineal descrito en el Algoritmo 3 
de Clase 19 usando la fórmula de Fletcher-Reeves:

$$ \beta_{k+1} = \frac{\nabla f_{k+1}^\top \nabla f_{k+1}}{\nabla f_{k}^\top\nabla f_{k}}  $$ 

```


```

1. Escriba la función que implemente el algoritmo. 

- La función debe recibir como argumentos $\mathbf{x}_0$, la función $f$ y 
  su gradiente, el número máximo de iteraciones $N$, la tolerancia $\tau$, y los
  parámetros para el algoritmo de backtracking: factor $\rho$, la constante $c_1$
  para la condición de descenso suficiente, la constante $c_2$ para la condición
  de curvatura, y el máximo número de iteraciones $N_b$.
- Agregue al algoritmo un contador
  $nr$ que se incremente cada vez que se aplique el reinicio, es decir, cuando
  se hace $\beta_{k+1}=0$.
   
- Para calcular el tamaño de paso $\alpha_k$ use el algoritmo de backtracking
  usando las condiciones de Wolfe con el valor inicial $\alpha_{ini}=1$.

- Haga que la función devuelva el último punto  $\mathbf{x}_k$, 
  el último gradiente $\mathbf{g}_k$, el número de iteraciones $k$ 
  y una variable binaria $bres$ que indique si se cumpli\'o el criterio
  de paro ($bres=True$) o si el algoritmo terminó por
  iteraciones ($bres=False$), y el contador $br·.

2. Pruebe el algoritmo usando la siguientes funciones con los puntos iniciales dados:


**Función de cuadrática 1:** Para $\mathbf{x}=(x_1,x_2, ..., x_n)$

- $f(\mathbf{x}) = \frac{1}{2} \mathbf{x}^\top\mathbf{A}_1\mathbf{x} - \mathbf{b}_1^\top\mathbf{x}$,
  donde $\mathbf{A}_1$ y $\mathbf{b}_1$ están definidas como en el Ejercicio 1.
- $\mathbf{x}_0 = (0,...,0)\in \mathbb{R}^{10}$ 
- $\mathbf{x}_0 = (0,...,0)\in \mathbb{R}^{100}$ 
- $\mathbf{x}_0 = (0,...,0)\in \mathbb{R}^{1000}$ 

**Función de cuadrática 2:** Para $\mathbf{x}=(x_1,x_2, ..., x_n)$

- $f(\mathbf{x}) = \frac{1}{2} \mathbf{x}^\top\mathbf{A}_2\mathbf{x} - \mathbf{b}_2^\top\mathbf{x}$,
  donde $\mathbf{A}_2$ y $\mathbf{b}_2$ están definidas como en el Ejercicio 1.
- $\mathbf{x}_0 = (0,...,0)\in \mathbb{R}^{10}$ 
- $\mathbf{x}_0 = (0,...,0)\in \mathbb{R}^{100}$ 
- $\mathbf{x}_0 = (0,...,0)\in \mathbb{R}^{1000}$ 

**Función de Beale :** Para $\mathbf{x}=(x_1,x_2)$

$$f(\mathbf{x}) = (1.5-x_1 + x_1x_2)^2 + (2.25 - x_1 + x_1x_2^2)^2 + (2.625 - x_1 + x_1x_2^3)^2.$$
- $\mathbf{x}_0 = (2,3)$  
   

**Función de Himmelblau:** Para $\mathbf{x}=(x_1,x_2)$

$$f(\mathbf{x}) = (x_1^2 + x_2 - 11)^2 + (x_1 + x_2^2 - 7)^2. $$
- $\mathbf{x}_0 = (2,4)$



**Función de Rosenbrock:** Para $\mathbf{x}=(x_1,x_2, ..., x_n)$

$$ f(\mathbf{x}) = \sum_{i=1}^{n-1} \left[100(x_{i+1} - x_i^2)^2 + (1-x_i)^2 \right]
\quad n\geq 2.$$
- $\mathbf{x}_0 = (-1.2, 1.0)\in \mathbb{R}^{2}$  
- $\mathbf{x}_0 = (-1.2, 1.0, ..., -1.2, 1.0) \in \mathbb{R}^{20}$  
- $\mathbf{x}_0 = (-1.2, 1.0, ..., -1.2, 1.0) \in \mathbb{R}^{40}$ 


3. Fije $N=5000$, $\tau = \sqrt{n}\epsilon_m^{1/3}$, donde $n$ es la dimensión
   de la variable $\mathbf{x}$ y $\epsilon_m$ es el épsilon máquina. 
   Para backtracking use $\rho=0.5$, $c_1=0.001$, $c_2=0.01$, $N_b=500$.
   
4. Para cada función de prueba imprima
   
- la dimensión $n$,
- $f(\mathbf{x}_0)$,
- el  número $k$ de iteraciones realizadas,
- $f(\mathbf{x}_k)$,
- las primeras y últimas 4 entradas del punto $\mathbf{x}_k$ que devuelve el algoritmo,
- la norma del vector gradiente $\mathbf{g}_k$, 
- la variable $bres$ para saber si el algoritmo puedo converger.
- el número de reinicios $nr$.
  

### Solución:

In [7]:
def nonlinear_conjugate_gradient( x0:np.ndarray, 
                                  f:Callable[[np.ndarray], float],
                                  grad_f:Callable[[np.ndarray], np.ndarray],
                                  max_iter:int,
                                  tol:float,
                                  rho:float=0.5,
                                  c1:float=1e-4, 
                                  c2:float=0.9,
                                  max_iter_b:int=100,
                                  alt = False) -> Tuple[np.ndarray, np.ndarray, int, bool]: 
                                  
    """
    Conjugate gradient method to find the minimum of a function
    
    :param x0: initial guess (np.ndarray)
    :param f: function to minimize (Callable[[np.ndarray], float])
    :param grad_f: gradient of the function (Callable[[np.ndarray], np.ndarray])
    :param max_iter: maximum number of iterations (int)
    :param tol: tolerance (float)
    :param rho: parameter for the backtracking line search (float)
    :param c1: parameter for the Armijo condition (float)
    :param c2: parameter for the Wolfe condition (float)
    :param max_iter_b: maximum number of iterations for the backtracking line search (int)
    
    :return: Tuple with the minimum of the function, the gradient at the minimum, the number of iterations and a boolean indicating if the method converges
    """
    
    br = 0
    
    g0 = grad_f(x0)
    d0 = -g0
    for i in range(max_iter):
        if np.linalg.norm(g0) < tol:
            return x0, g0, i, True, br
        alpha,_ = opti.brack_tracking_Wolfe(alpha_init=1, rho=rho, c1=c1, c2=c2, xk=x0, f=f, gradf=grad_f, fk=f(x0), grad_fk=grad_f(x0), dir_pk=d0,iter_maxb=max_iter_b,alt=alt)
        xk = x0 + alpha * d0
        gk = grad_f(xk)
        if abs(gk.T@g0) < 0.2*gk.T@gk:
            beta = (gk.T @ gk) / (g0.T@g0)
        else: 
            beta = 0
            br+=1   
        dk = -gk + beta * d0
        x0 = xk
        g0 = gk
        d0 = dk
    return x0, g0, i, False, br

2. Testear el algoritmo en las funciones:

In [8]:
def print_non_linear_conjugate_gradient(f:Callable[[np.ndarray], float],
                                        grad_f:Callable[[np.ndarray], np.ndarray],
                                        x0:np.ndarray, 
                                        max_iter:int, 
                                        tol:float, 
                                        rho:float, 
                                        c1:float, 
                                        c2:float, 
                                        N_b:int,
                                        alt:bool=False) -> None:
    xk, grad_xk, iterk, conv, br = nonlinear_conjugate_gradient(x0,f, grad_f, max_iter, tol, rho, c1, c2, N_b,alt)

    print("Dimensión:\t\t",len(x0))
    print("f(x0):\t\t\t",f(x0))
    print("Número de iteraciones:\t",iterk)
    print("f(xk):\t\t\t",f(xk))
    if n>4:
        print("xk:\t\t",','.join(map(str, xk[:4])),",...,",','.join(map(str, xk[-4:])))
    else:
        print("xk:\t\t\t",xk)
    print("||grad_f(x_k)||: \t",np.linalg.norm(grad_xk))
    print("Convergencia:\t\t",conv)
    print("nr:\t\t\t",br)

Parametros:

In [9]:
max_iter = 5000 
rho = 0.5
c1 = 1e-3
c2 = 0.01
N_b = 500

**Función Cuadratica 1**

In [10]:
A11 = matrix1(10)
A12 = matrix1(100)
A13 = matrix1(1000)
b1 = np.ones(10)
b2 = np.ones(100)
b3 = np.ones(1000) 

f_square11 = lambda x: 0.5*x.T@A11@x-b1@x
gradf_square11 = lambda x: A11@x-b1

f_square12 = lambda x: 0.5*x.T@A12@x-b2@x
gradf_square12 = lambda x: A12@x-b2

f_square13 = lambda x: 0.5*x.T@A13@x-b3@x
gradf_square13 = lambda x: A13@x-b3

In [12]:
x0 = np.zeros(10)
n = len(x0)
tol = np.sqrt(n)*(np.finfo(float).eps)**(1/3)
print_non_linear_conjugate_gradient(f_square11,gradf_square11,x0,max_iter,tol,rho,c1,c2,N_b)

Dimensión:		 10
f(x0):			 0.0
Número de iteraciones:	 9
f(xk):			 -0.24999999999636202
xk:		 0.05000019073486328,0.05000019073486328,0.05000019073486328,0.05000019073486328 ,..., 0.05000019073486328,0.05000019073486328,0.05000019073486328,0.05000019073486328
||grad_f(x_k)||: 	 1.206313194339134e-05
Convergencia:		 True
nr:			 9


In [13]:
x0 = np.zeros(100)
n = len(x0)
tol = np.sqrt(n)*(np.finfo(float).eps)**(1/3)
print_non_linear_conjugate_gradient(f_square12,gradf_square12,x0,max_iter,tol,rho,c1,c2,N_b)

Dimensión:		 100
f(x0):			 0.0
Número de iteraciones:	 21
f(xk):			 -0.2499999999919999
xk:		 0.005000028284145768,0.005000028284145768,0.005000028284145768,0.005000028284145768 ,..., 0.005000028284145769,0.005000028284145769,0.005000028284145769,0.005000028284145769
||grad_f(x_k)||: 	 5.6568291540037836e-05
Convergencia:		 True
nr:			 21


In [14]:
x0 = np.zeros(1000)
n = len(x0)
tol = np.sqrt(n)*(np.finfo(float).eps)**(1/3)
print_non_linear_conjugate_gradient(f_square13,gradf_square13,x0,max_iter,tol,rho,c1,c2,N_b)

Dimensión:		 1000
f(x0):			 0.0
Número de iteraciones:	 251
f(xk):			 -0.2499999999914645
xk:		 0.0005000029213602828,0.0005000029213602828,0.0005000029213602828,0.0005000029213602828 ,..., 0.0005000029213602874,0.0005000029213602885,0.0005000029213602883,0.0005000029213602873
||grad_f(x_k)||: 	 0.00018476304752628408
Convergencia:		 True
nr:			 251


**Función Cuadratica 2**

In [15]:
A11 = matrix2(10)
A12 = matrix2(100)
A13 = matrix2(1000)
b1 = np.ones(10)
b2 = np.ones(100)
b3 = np.ones(1000)

f_square11 = lambda x: 0.5*x.T@A11@x-b1@x
gradf_square11 = lambda x: A11@x-b1

f_square12 = lambda x: 0.5*x.T@A12@x-b2@x
gradf_square12 = lambda x: A12@x-b2

f_square13 = lambda x: 0.5*x.T@A13@x-b3@x
gradf_square13 = lambda x: A13@x-b3

In [16]:
x0 = np.zeros(10)
n = len(x0)
tol = np.sqrt(n)*(np.finfo(float).eps)**(1/3)
print_non_linear_conjugate_gradient(f_square11,gradf_square11,x0,max_iter,tol,rho,c1,c2,N_b)

Dimensión:		 10
f(x0):			 0.0
Número de iteraciones:	 1571
f(xk):			 -1.7934207913526619
xk:		 1.3688956634167995,-1.165862919253239,1.6083890463638972,-0.6127911466676729 ,..., -0.6127911466676721,1.6083890463638963,-1.1658629192532382,1.3688956634167995
||grad_f(x_k)||: 	 1.895048303749792e-05
Convergencia:		 True
nr:			 1230


In [17]:
rho = 0.6
x0 = np.zeros(100)
n = len(x0)
tol = np.sqrt(n)*(np.finfo(float).eps)**(1/3)
print_non_linear_conjugate_gradient(f_square12,gradf_square12,x0,max_iter,tol,rho,c1,c2,N_b)

Dimensión:		 100
f(x0):			 0.0
Número de iteraciones:	 4999
f(xk):			 -14.494237120734711
xk:		 1.4394380697559654,-1.3950315645431932,2.069993316201003,-1.3637847328940806 ,..., -1.3637847328940682,2.0699933162009954,-1.3950315645431886,1.4394380697559637
||grad_f(x_k)||: 	 0.0010685994764851967
Convergencia:		 False
nr:			 4981


In [25]:
x0 = np.zeros(1000)
n = len(x0)
tol = np.sqrt(n)*(np.finfo(float).eps)**(1/3)
rho = 0.5
print_non_linear_conjugate_gradient(f_square13,gradf_square13,x0,max_iter,tol,rho,c1,c2,N_b,alt=True)

Dimensión:		 1000
f(x0):			 0.0
Número de iteraciones:	 2590
f(xk):			 -141.4369672302889
xk:		 1.4436347652581003,-1.4080703063030438,2.0946540196166907,-1.4008840724651848 ,..., -1.400884072465163,2.0946540196166756,-1.4080703063030353,1.4436347652580972
||grad_f(x_k)||: 	 0.00018992149007912416
Convergencia:		 True
nr:			 2103


**Función Himmelblau**

In [19]:
x0 = np.array([2.0,4.0])
n = len(x0)
tol = np.sqrt(n)*(np.finfo(float).eps)**(1/3)
print_non_linear_conjugate_gradient(opti.fncHimmelblau,opti.grad_fncHimmelblau,x0,max_iter,tol,rho,c1,c2,N_b)

Dimensión:		 2
f(x0):			 130.0
Número de iteraciones:	 31
f(xk):			 2.4720276275992285e-13
xk:			 [ 3.58442828 -1.84812647]
||grad_f(x_k)||: 	 6.528645259773077e-06
Convergencia:		 True
nr:			 31


**Función Beale:**

In [20]:
x0 = np.array([2.0,3.0])
n = len(x0)
tol = np.sqrt(n)*(np.finfo(float).eps)**(1/3)
print_non_linear_conjugate_gradient(opti.fncBeale, opti.grad_fncBeale,x0,max_iter,tol,rho,c1,c2,N_b)

Dimensión:		 2
f(x0):			 3347.203125
Número de iteraciones:	 1619
f(xk):			 2.2029216391375518e-11
xk:			 [3.00001161 0.50000274]
||grad_f(x_k)||: 	 8.380681550427131e-06
Convergencia:		 True
nr:			 1619


**Función Rosenbrock** 

In [31]:
x0 = np.array([-1.2,1.0])
n = len(x0)
tol = np.sqrt(n)*(np.finfo(float).eps)**(1/3)
print_non_linear_conjugate_gradient(opti.fncRosenbrock, opti.grad_fncRosenbrock,x0,max_iter,tol,rho,c1,c2,N_b)

Dimensión:		 2
f(x0):			 24.199999999999996
Número de iteraciones:	 4999
f(xk):			 7.919067457475048e-10
xk:			 [1.00002802 1.0000563 ]
||grad_f(x_k)||: 	 7.000359375329152e-05
Convergencia:		 False
nr:			 4248


In [28]:
x0 = np.array([-1.2,1.0]*10)
n = len(x0)
tol = np.sqrt(n)*(np.finfo(float).eps)**(1/3)
print_non_linear_conjugate_gradient(opti.fncRosenbrock, opti.grad_fncRosenbrock,x0,max_iter,tol,rho,c1,c2,N_b)

Dimensión:		 20
f(x0):			 4598.0
Número de iteraciones:	 860
f(xk):			 1.0453665104406672e-12
xk:		 1.0000000045877049,1.0000000053488944,1.0000000097468837,1.000000008128462 ,..., 1.000000180660494,1.000000349774317,1.000000699531955,1.0000013983356322
||grad_f(x_k)||: 	 2.4641847474138866e-05
Convergencia:		 True
nr:			 692


In [36]:
x0 = np.array([-1.2,1.0]*20)
n = len(x0)
rho = 0.6
tol = np.sqrt(n)*(np.finfo(float).eps)**(1/3)
print_non_linear_conjugate_gradient(opti.fncRosenbrock, opti.grad_fncRosenbrock,x0,max_iter,tol,rho,c1,c2,N_b)

Dimensión:		 40
f(x0):			 9680.000000000004
Número de iteraciones:	 2337
f(xk):			 1.4191714098163482e-10
xk:		 0.9999999997589897,1.0000000006125462,0.9999999990394711,1.000000001327496 ,..., 1.0000025616005486,1.0000051384392061,1.0000103005292058,1.000020653404749
||grad_f(x_k)||: 	 3.598898991128314e-05
Convergencia:		 True
nr:			 2228



```


```
---

# <font color="8C3061" >**Ejercicio 3 (3.5 puntos)**</font>

Programar el método de gradiente conjugado no lineal de usando la fórmula de
Hestenes-Stiefel:

En este caso el algoritmo es igual al del Ejercicio 2, con excepción del cálculo de $\beta_{k+1}$. Primero se calcula el vector $\mathbf{y}_k$ y luego $\beta_{k+1}$:

$$ \mathbf{y}_k =  \nabla f_{k+1}-\nabla f_{k} $$
$$ \beta_{k+1} =   \frac{\nabla f_{k+1}^\top\mathbf{y}_k }{\nabla p_{k}^\top\mathbf{y}_k}  $$

1. Repita el Ejercicio 2 usando la fórmula de Hestenes-Stiefel.
2. ¿Hay alguna diferencia que indique que es mejor usar la fórmula de Hestenes-Stiefel
   respesto a Fletcher-Reeves?
3. La cantidad de reinicios puede indicar que tanto se comporta el algoritmo
   como el algoritmo de descenso máximo. Agregue un comentario sobre esto 
   de acuerdo a los resultados obtenidos para cada fórmula.

### Solución:

<div class="alert alert-block alert-info">
<b>Nota:</b> La implementación de backtraking se realizó de dos maneras, una en la que busca encontrar el tamaño de paso tal que se satisfaga las condiciones de Wolfe, como es indicado en la tarea. Además, se considera una alternativa que busca hallar paso de salto que cumpla la condición de descenso suficiente y la de curvatura, pero, en caso de no cumplir la de curvatura, seleccione el primer tamaño de paso que cumple Armijo con el fin de no dar pasos tan pequeños con frecuencia.

Además, con el fin de que el código corriera más rápido se asignó $\rho=0.6$, con el fin de que aumentase la probabilidad de no recorrer todo el algoritmo de backtracking, ya que $alpha$ tiene más puntos para analizar que puedan cumplir las condiciones de curvatura.  </div>
</div>

In [57]:
def nonlinear_conjugate_gradient2(x0:np.ndarray, 
                                  f:Callable[[np.ndarray], float],
                                    grad_f:Callable[[np.ndarray], np.ndarray],
                                    max_iter:int,
                                    tol:float,
                                    rho:float=0.5,
                                    c1:float=1e-4,
                                    c2:float=0.9,
                                    max_iter_b:int=100,
                                    alt = False) -> Tuple[np.ndarray, np.ndarray, int, bool, int]: 
                                  
    """
    Conjugate gradient method to find the minimum of a function
    
    :param x0: initial guess
    :param f: function to minimize
    :param grad_f: gradient of the function
    :param max_iter: maximum number of iterations
    :param tol: tolerance
    :param rho: parameter for the backtracking line search
    :param c1: parameter for the Armijo condition
    :param c2: parameter for the Wolfe condition
    :param max_iter_b: maximum number of iterations for the backtracking line search
    
    :return: Tuple with the minimum of the function, the gradient at the minimum, the number of iterations and a boolean indicating if the method converges
    """
    br = 0
    
    g0 = grad_f(x0)
    d0 = -g0
    for i in range(max_iter):
        if np.linalg.norm(g0) < tol:
            return x0, g0, i, True, br
        
        alpha,_ = opti.brack_tracking_Wolfe(alpha_init=1, rho=rho, c1=c1, c2=c2, xk=x0, f=f, gradf=grad_f, fk=f(x0), grad_fk=grad_f(x0), dir_pk=d0,iter_maxb=max_iter_b,alt=alt)
        xk = x0 + alpha * d0
        gk = grad_f(xk)
        if abs(gk.T@g0) < 0.2*gk.T@gk:
            yk = gk - g0
            beta = (gk.T @ yk) / (d0.T @ yk) 
        else: 
            beta = 0
            br+=1   
        dk = -gk + beta * d0
        x0 = xk
        g0 = gk
        d0 = dk
    return x0, g0, i, False, br

In [58]:
def print_non_linear_conjugate_gradient2(f:Callable[[np.ndarray], float],
                                        grad_f:Callable[[np.ndarray], np.ndarray],
                                        x0:np.ndarray, 
                                        max_iter:int, 
                                        tol:float, 
                                        rho:float, 
                                        c1:float, 
                                        c2:float, 
                                        N_b:int,
                                        alt:bool=False) -> None:
    xk, grad_xk, iterk, conv, br = nonlinear_conjugate_gradient2(x0,f, grad_f, max_iter, tol, rho, c1, c2, N_b,alt)

    print("Dimensión:\t\t",len(x0))
    print("f(x0):\t\t\t",f(x0))
    print("Número de iteraciones:\t",iterk)
    print("f(xk):\t\t\t",f(xk))
    if n>4:
        print("xk:\t\t",','.join(map(str, xk[:4])),",...,",','.join(map(str, xk[-4:])))
    else:
        print("xk:\t\t\t",xk)
    print("||grad_f(x_k)||: \t",np.linalg.norm(grad_xk))
    print("Convergencia:\t\t",conv)
    print("nr:\t\t\t",br)

In [59]:
max_iter = 5000 
rho = 0.5
c1 = 1e-3
c2 = 0.01
N_b = 500

**Función Cuadratica 1**


In [60]:
A11 = matrix1(10)
A12 = matrix1(100)
A13 = matrix1(1000)
b1 = np.ones(10)
b2 = np.ones(100)
b3 = np.ones(1000) 
f_square11 = lambda x: 0.5*x.T@A11@x-b1@x
gradf_square11 = lambda x: A11@x-b1

f_square12 = lambda x: 0.5*x.T@A12@x-b2@x
gradf_square12 = lambda x: A12@x-b2

f_square13 = lambda x: 0.5*x.T@A13@x-b3@x
gradf_square13 = lambda x: A13@x-b3

In [61]:
x0 = np.zeros(10)
n = len(x0)
tol = np.sqrt(n)*(np.finfo(float).eps)**(1/3)
print_non_linear_conjugate_gradient2(f_square11,gradf_square11,x0,max_iter,tol,rho,c1,c2,N_b)

Dimensión:		 10
f(x0):			 0.0
Número de iteraciones:	 9
f(xk):			 -0.24999999999636202
xk:		 0.05000019073486328,0.05000019073486328,0.05000019073486328,0.05000019073486328 ,..., 0.05000019073486328,0.05000019073486328,0.05000019073486328,0.05000019073486328
||grad_f(x_k)||: 	 1.206313194339134e-05
Convergencia:		 True
nr:			 9


In [62]:
x0 = np.zeros(100)
n = len(x0)
tol = np.sqrt(n)*(np.finfo(float).eps)**(1/3)
print_non_linear_conjugate_gradient2(f_square12,gradf_square12,x0,max_iter,tol,rho,c1,c2,N_b)

Dimensión:		 100
f(x0):			 0.0
Número de iteraciones:	 21
f(xk):			 -0.2499999999919999
xk:		 0.005000028284145768,0.005000028284145768,0.005000028284145768,0.005000028284145768 ,..., 0.005000028284145769,0.005000028284145769,0.005000028284145769,0.005000028284145769
||grad_f(x_k)||: 	 5.6568291540037836e-05
Convergencia:		 True
nr:			 21


In [63]:
x0 = np.zeros(1000)
n = len(x0)
tol = np.sqrt(n)*(np.finfo(float).eps)**(1/3)
print_non_linear_conjugate_gradient2(f_square13,gradf_square13,x0,max_iter,tol,rho,c1,c2,N_b)

Dimensión:		 1000
f(x0):			 0.0
Número de iteraciones:	 251
f(xk):			 -0.2499999999914645
xk:		 0.0005000029213602828,0.0005000029213602828,0.0005000029213602828,0.0005000029213602828 ,..., 0.0005000029213602874,0.0005000029213602885,0.0005000029213602883,0.0005000029213602873
||grad_f(x_k)||: 	 0.00018476304752628408
Convergencia:		 True
nr:			 251


**Función Cuadratica 2**

In [64]:
A11 = matrix2(10)
A12 = matrix2(100)
A13 = matrix2(1000)
b1 = np.ones(10)
b2 = np.ones(100)
b3 = np.ones(1000)

f_square11 = lambda x: 0.5*x.T@A11@x-b1@x
gradf_square11 = lambda x: A11@x-b1

f_square12 = lambda x: 0.5*x.T@A12@x-b2@x
gradf_square12 = lambda x: A12@x-b2

f_square13 = lambda x: 0.5*x.T@A13@x-b3@x
gradf_square13 = lambda x: A13@x-b3

In [65]:
x0 = np.zeros(10)
n = len(x0)
tol = np.sqrt(n)*(np.finfo(float).eps)**(1/3)
print_non_linear_conjugate_gradient2(f_square11,gradf_square11,x0,max_iter,tol,rho,c1,c2,N_b)

Dimensión:		 10
f(x0):			 0.0
Número de iteraciones:	 1571
f(xk):			 -1.7934207913526619
xk:		 1.3688956634167995,-1.165862919253239,1.6083890463638972,-0.6127911466676729 ,..., -0.6127911466676721,1.6083890463638963,-1.1658629192532382,1.3688956634167995
||grad_f(x_k)||: 	 1.895048303749792e-05
Convergencia:		 True
nr:			 1230


In [66]:
rho = 0.6
x0 = np.zeros(100)
n = len(x0)
tol = np.sqrt(n)*(np.finfo(float).eps)**(1/3)
print_non_linear_conjugate_gradient2(f_square12,gradf_square12,x0,max_iter,tol,rho,c1,c2,N_b)

Dimensión:		 100
f(x0):			 0.0
Número de iteraciones:	 4999
f(xk):			 -14.494237120734711
xk:		 1.4394380697559654,-1.3950315645431932,2.069993316201003,-1.3637847328940806 ,..., -1.3637847328940682,2.0699933162009954,-1.3950315645431886,1.4394380697559637
||grad_f(x_k)||: 	 0.0010685994764851967
Convergencia:		 False
nr:			 4981


In [67]:
x0 = np.zeros(1000)
n = len(x0)
tol = np.sqrt(n)*(np.finfo(float).eps)**(1/3)
rho = 0.5
print_non_linear_conjugate_gradient2(f_square13,gradf_square13,x0,max_iter,tol,rho,c1,c2,N_b,alt=True)

Dimensión:		 1000
f(x0):			 0.0
Número de iteraciones:	 4999
f(xk):			 -141.43691110446827
xk:		 1.4404743792219015,-1.398298401648737,2.076050769403433,-1.3728570552095603 ,..., -1.3728570552095452,2.076050769403422,-1.3982984016487308,1.4404743792219
||grad_f(x_k)||: 	 0.0005855903072393323
Convergencia:		 False
nr:			 3720


**Función Himmelblau**

In [68]:
x0 = np.array([2.0,4.0])
n = len(x0)
tol = np.sqrt(n)*(np.finfo(float).eps)**(1/3)
print_non_linear_conjugate_gradient2(opti.fncHimmelblau,opti.grad_fncHimmelblau,x0,max_iter,tol,rho,c1,c2,N_b)

Dimensión:		 2
f(x0):			 130.0
Número de iteraciones:	 37
f(xk):			 1.9833788488942771e-13
xk:			 [ 3.58442828 -1.84812653]
||grad_f(x_k)||: 	 6.466611484709132e-06
Convergencia:		 True
nr:			 36


**Función Beale:**

In [69]:
x0 = np.array([2.0,3.0])
n = len(x0)
tol = np.sqrt(n)*(np.finfo(float).eps)**(1/3)
print_non_linear_conjugate_gradient2(opti.fncBeale, opti.grad_fncBeale,x0,max_iter,tol,rho,c1,c2,N_b)

Dimensión:		 2
f(x0):			 3347.203125
Número de iteraciones:	 769
f(xk):			 5.6113870648033834e-11
xk:			 [3.00001871 0.50000457]
||grad_f(x_k)||: 	 7.496323664379619e-06
Convergencia:		 True
nr:			 585


**Función Rosenbrock** 

In [70]:
x0 = np.array([-1.2,1.0])
n = len(x0)
tol = np.sqrt(n)*(np.finfo(float).eps)**(1/3)
print_non_linear_conjugate_gradient2(opti.fncRosenbrock, opti.grad_fncRosenbrock,x0,max_iter,tol,rho,c1,c2,N_b)

Dimensión:		 2
f(x0):			 24.199999999999996
Número de iteraciones:	 1382
f(xk):			 1.606990811512821e-11
xk:			 [0.999996   0.99999198]
||grad_f(x_k)||: 	 8.384827957465905e-06
Convergencia:		 True
nr:			 1128


In [71]:
x0 = np.array([-1.2,1.0]*10)
n = len(x0)
tol = np.sqrt(n)*(np.finfo(float).eps)**(1/3)
print_non_linear_conjugate_gradient2(opti.fncRosenbrock, opti.grad_fncRosenbrock,x0,max_iter,tol,rho,c1,c2,N_b)

Dimensión:		 20
f(x0):			 4598.0
Número de iteraciones:	 1615
f(xk):			 8.876616970657303e-11
xk:		 1.0000000032570238,1.0000000115016339,1.0000000004616163,1.0000000105180396 ,..., 1.0000020219610348,1.0000040577194274,1.0000081453807228,1.0000163324635707
||grad_f(x_k)||: 	 2.657427171845461e-05
Convergencia:		 True
nr:			 1265


In [72]:
x0 = np.array([-1.2,1.0]*20)
n = len(x0)
rho = 0.6
tol = np.sqrt(n)*(np.finfo(float).eps)**(1/3)
print_non_linear_conjugate_gradient2(opti.fncRosenbrock, opti.grad_fncRosenbrock,x0,max_iter,tol,rho,c1,c2,N_b)

Dimensión:		 40
f(x0):			 9680.000000000004
Número de iteraciones:	 1347
f(xk):			 2.6581720338929803e-11
xk:		 1.0000000002214249,0.9999999994435402,1.0000000008515648,0.9999999988722171 ,..., 1.000001080190785,1.0000021798670349,1.0000043859552445,1.0000087995265188
||grad_f(x_k)||: 	 3.349524293049548e-05
Convergencia:		 True
nr:			 1222


1. No se evidencia alguna mejora o empeoramiento usando ambas formulas, ya que bajo unas funciones funciona mejor una formula que la otra pero no se evidencia una mejora substancial. Además que se observa que bajo estos algoritmos son bastantes sensibles a la parametrización del backtracking con las condiciones de Wolfe.

2.  El número de reinicios indica que en la mayor parte del tiempo el algoritmo se comporta como el algoritmo de descenso máximo.